<a href="https://colab.research.google.com/github/rifkybujana/KOPSI/blob/Research%2FModel/DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INSTALLATION

In [1]:
!pip install tokenizers

     |████████████████████████████████| 3.3MB 5.3MB/s 


In [2]:
!pip install transformers

     |████████████████████████████████| 2.5MB 5.8MB/s 
     |████████████████████████████████| 901kB 39.3MB/s 


## CODE

Import required libraries

In [12]:
import torch
import pandas as pd
import numpy as np

from ast import literal_eval

from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering

from transformers import AdamW

from torch.utils.data import DataLoader

### Analyzing FacQA dataset

import train and eval dataset

In [7]:
train = pd.read_csv('train_preprocess.csv')
valid = pd.read_csv('val_preprocess.csv')

analalyze train dataset

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2495 entries, 0 to 2494
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   question   2495 non-null   object
 1   passage    2495 non-null   object
 2   seq_label  2495 non-null   object
dtypes: object(3)
memory usage: 58.6+ KB


In [9]:
train.describe()

,question,passage,seq_label
count,2495,2495,2495
unique,2412,1216,2077
top,"['Siapa', 'nama', 'presiden', 'Indonesia', 'se...","['Pemilu', 'pertama', 'multikandidat', 'ini', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
freq,7,17,6


In [8]:
train.head()

,question,passage,seq_label
0,"['Kelompok', 'apakah', 'yang', 'menyatakan', '...","['Lewat', 'telepon', 'ke', 'kantor', 'berita',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"['Di', 'pulau', 'manakah', ',', 'PT', 'Mutiara...","['Sejak', 'tahun', '1980', '-', 'an', 'di', 'K...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['Apakah', 'warna', 'pasir', 'pada', 'pesisir'...","['Titik', 'kedua', 'adalah', 'Pantai', 'Baturu...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Apa', 'nama', 'perusahaan', 'China', 'yang',...","['Sichuan', ',', 'Kompas', '-', 'Pemerintah', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['Siapa', 'yang', 'akan', 'memindahkan', 'Patu...","['Jakarta', ',', 'Kompas', '-', 'Patung', 'Kar...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [11]:
type(train['question'][0])

str

As you can see, the dataset data type are string, but we want them to be a list, not a string, we need to remove all the brackets, quotation marks, and the coma's. separate every word, and convert it into an array using `literal_eval` function

In [14]:
train['question'] = train['question'].apply(literal_eval)
train['passage'] = train['passage'].apply(literal_eval)
train['seq_label'] = train['seq_label'].apply(literal_eval)

In [16]:
valid['question'] = valid['question'].apply(literal_eval)
valid['passage'] = valid['passage'].apply(literal_eval)
valid['seq_label'] = valid['seq_label'].apply(literal_eval)

after applied all the dataset values with the `literal_eval` function, we can see, that the dataset datatype changed into a list

In [18]:
print(f"""train dataset data type: {type(train['question'][0])}""")
print(f"""valid dataset data type: {type(valid['question'][0])}""")

train dataset data type: <class 'list'>
valid dataset data type: <class 'list'>
